In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00


In [54]:
import torch
import pandas as pd
from transformers import pipeline
from datasets import load_dataset

###Task: Machine Reading Comprehension###

####1. Data preparation (10 points)####

**1) (5 points) Data download and preprocessing:**

In [3]:
dataset = load_dataset("squad_v2")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [5]:
data = pd.DataFrame(dataset['train'])

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130319 entries, 0 to 130318
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        130319 non-null  object
 1   title     130319 non-null  object
 2   context   130319 non-null  object
 3   question  130319 non-null  object
 4   answers   130319 non-null  object
dtypes: object(5)
memory usage: 5.0+ MB


In [7]:
data

,id,title,context,question,answers
0,56be85543aeaaa14008c9063,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start'..."
1,56be85543aeaaa14008c9065,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,"{'text': ['singing and dancing'], 'answer_star..."
2,56be85543aeaaa14008c9066,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,"{'text': ['2003'], 'answer_start': [526]}"
3,56bf6b0f3aeaaa14008c9601,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"{'text': ['Houston, Texas'], 'answer_start': [..."
4,56bf6b0f3aeaaa14008c9602,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,"{'text': ['late 1990s'], 'answer_start': [276]}"
...,...,...,...,...,...
130314,5a7e070b70df9f001a875439,Matter,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,"{'text': [], 'answer_start': []}"
130315,5a7e070b70df9f001a87543a,Matter,"The term ""matter"" is used throughout physics i...",Who coined the term partonic matter?,"{'text': [], 'answer_start': []}"
130316,5a7e070b70df9f001a87543b,Matter,"The term ""matter"" is used throughout physics i...",What is another name for anti-matter?,"{'text': [], 'answer_start': []}"
130317,5a7e070b70df9f001a87543c,Matter,"The term ""matter"" is used throughout physics i...",Matter usually does not need to be used in con...,"{'text': [], 'answer_start': []}"


identifying indixes with missing data in 'answers'['text']

In [8]:
ind = [i for i, v in enumerate(data['answers']) if not v['text']]

In [9]:
data.loc[2075]

id                                   5a8d7bf7df8bba001a0f9ab1
title                  The_Legend_of_Zelda:_Twilight_Princess
context     The Legend of Zelda: Twilight Princess (Japane...
question    What category of game is Legend of Zelda: Aust...
answers                      {'text': [], 'answer_start': []}
Name: 2075, dtype: object

dropping these indices

In [10]:
data = data.drop(index=ind)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86821 entries, 0 to 130050
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        86821 non-null  object
 1   title     86821 non-null  object
 2   context   86821 non-null  object
 3   question  86821 non-null  object
 4   answers   86821 non-null  object
dtypes: object(5)
memory usage: 4.0+ MB


selecting sample of 15 question-answer pairs

In [12]:
select = data.sample(n=15, random_state=1234).reset_index(drop=True)

In [70]:
select.head()

,id,title,context,question,answers
0,56f82a24aef2371900625e78,Josip_Broz_Tito,"His next relationship was with Herta Haas, who...",With whom did Tito have a parallel life during...,"{'text': ['Davorjanka Paunović'], 'answer_star..."
1,56db5ff1e7c41114004b5025,American_Idol,Various American Idol alumni had success on va...,How many radio spins did Kelly Clarkson have b...,"{'text': ['over four million'], 'answer_start'..."
2,5727c9722ca10214002d9634,USB,Firstly conceived and still used today for opt...,What is the most recent standard for external ...,"{'text': ['Thunderbolt'], 'answer_start': [784]}"
3,56dda6db66d3e219004dac94,Institute_of_technology,The institutes of technology and polytechnics ...,In what year was the École Polytechnique founded?,"{'text': ['1794'], 'answer_start': [840]}"
4,5727e8432ca10214002d993c,Oklahoma,State law codifies Oklahoma's state emblems an...,Why did the state House think they could chang...,"{'text': ['no ""official"" vote regarding ""Labor..."


**2) (5 points) Data statistics:**

In [14]:
context_len = [len(i) for i in select['context']]
print("The statistics of the 'contexts':\n")
print(f"maximum length:\t{max(context_len)}\naverage length:\t{sum(context_len)/len(context_len)}\nminimum length:\t{min(context_len)}\n\n")

ques_len = [len(i) for i in select['question']]
print("The statistics of the 'questions':\n")
print(f"maximum length:\t{max(ques_len)}\naverage length:\t{sum(ques_len)/len(ques_len)}\nminimum length:\t{min(ques_len)}\n\n")

ans_len = [len(i['text'][0]) for i in select['answers']]
print("The statistics of the 'answers':\n")
print(f"maximum length:\t{max(ans_len)}\naverage length:\t{sum(ans_len)/len(ans_len)}\nminimum length:\t{min(ans_len)}\n\n")

The statistics of the 'contexts':

maximum length:	1089
average length:	786.5333333333333
minimum length:	497


The statistics of the 'questions':

maximum length:	108
average length:	64.0
minimum length:	37


The statistics of the 'answers':

maximum length:	111
average length:	42.0
minimum length:	4




####2. Question answering (60 points, 20 points for each model)####

#####Pre-trained RoBERTa model#####

In [16]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [73]:
op = []
for i in select.iterrows():
  inputs = tokenizer(i[1]['question'], i[1]['context'], return_tensors="pt")
  with torch.no_grad():
      outputs = model(**inputs)

  answer_start_index = outputs.start_logits.argmax()
  answer_end_index = outputs.end_logits.argmax()

  predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
  decoded_answer = tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

  target_start_index = torch.tensor([14])
  target_end_index = torch.tensor([15])

  outputs = model(**inputs, start_positions=target_start_index, end_positions=target_end_index)
  op.append(outputs)
  loss = outputs.loss
  # print(round(loss.item(), 2))
  print(f"\n\nQuestion:\t{i[1]['question']}")
  print(f"\nAnswer Start Index: \n{outputs.start_logits}")
  print(f"\nAnswer End Index: \n{outputs.end_logits}")
  print(f"\nPredicted Answer Tokens: \n{outputs.start_logits}, {outputs.end_logits}")
  print(f"\nDecoded Answer:\t{decoded_answer}")
  # break



Question:	With whom did Tito have a parallel life during his marriage to Haas?

Answer Start Index: 
tensor([[ 0.7387, -8.9269, -9.3667, -9.4724, -8.4977, -9.5210, -9.2633, -9.3209,
         -9.1548, -9.3686, -9.1368, -8.7980, -8.8388, -9.1545, -8.9766, -9.6856,
         -9.8780, -9.9484, -6.5374, -8.0537, -7.2832, -8.3846, -6.8170, -1.1256,
         -7.6247, -3.7214, -8.7005, -8.5333, -7.6578, -6.9023, -7.4457, -4.9918,
         -9.8029, -3.3811, -7.4317, -8.7051, -8.9347, -6.0960, -8.0727, -8.6274,
         -8.2302, -7.5283, -8.3221, -9.6349, -7.9748, -5.1602, -8.0147, -4.9906,
         -8.0293, -7.3744, -7.0085, -9.8076, -6.4807, -8.6267, -8.9382, -9.0103,
         -6.9227, -8.5866, -9.5578, -6.4764, -7.9283, -7.6208, -8.5328, -8.4086,
         -6.2194, -7.8736, -8.0778, -9.3444, -5.2967, -8.0591, -4.9906, -3.3842,
         -5.5430, -4.7111, -5.7659, -6.6298, -2.7564, -6.1860,  0.0114, -5.4744,
         -5.3509, -3.9652, -3.7840, -2.4388, -5.8341, -5.7797, -5.5703, -4.1861,
      

In [67]:
# using the pipeline method
nlp = pipeline('question-answering', model="deepset/roberta-base-squad2", tokenizer="deepset/roberta-base-squad2")

for i in select.iterrows():
  inp = {
      'question':i[1]['question'],
      'context' :i[1]['context']
  }
  res = nlp(inp)
  print("\nData Sample Output:")
  print(f"Answer Start Index: {res['start']}")
  print(f"Answer End Index: {res['end']}")
  print(f"Decoded Answer: {res['answer']}")


Data Sample Output:
Answer Start Index: 320
Answer End Index: 339
Decoded Answer: Davorjanka Paunović

Data Sample Output:
Answer Start Index: 726
Answer End Index: 743
Decoded Answer: over four million

Data Sample Output:
Answer Start Index: 784
Answer End Index: 795
Decoded Answer: Thunderbolt

Data Sample Output:
Answer Start Index: 840
Answer End Index: 844
Decoded Answer: 1794

Data Sample Output:
Answer Start Index: 445
Answer End Index: 495
Decoded Answer: a constituent researched the Oklahoma Constitution

Data Sample Output:
Answer Start Index: 378
Answer End Index: 387
Decoded Answer: the Dutch

Data Sample Output:
Answer Start Index: 461
Answer End Index: 477
Decoded Answer: Nomadic Bedouins

Data Sample Output:
Answer Start Index: 235
Answer End Index: 267
Decoded Answer: to start a regular "Bible study"

Data Sample Output:
Answer Start Index: 134
Answer End Index: 180
Decoded Answer: the gene's DNA is transcribed to messenger RNA

Data Sample Output:
Answer Start Index:

#####ChatGPT#####

**Designing prompt**

Based on the following question and context, answer the question first as "**Question** : ", "**Answer** : " and then provide a brief explanation of your reasoning as "**Explanation** : ".

Question : ""

Context : ""

In [102]:
# for the given dataset
for i in select.iterrows():
  print(f"\nQuestion: {i[1]['question']}")
  print(f"Context: {i[1]['context']}")
  print(f"Answer: {i[1]['answers']}")
  # break


Question: With whom did Tito have a parallel life during his marriage to Haas?
Context: His next relationship was with Herta Haas, whom he married in 1940. Broz left for Belgrade after the April War, leaving Haas pregnant. In May 1941, she gave birth to their son, Aleksandar "Mišo" Broz. All throughout his relationship with Haas, Tito had maintained a promiscuous life and had a parallel relationship with Davorjanka Paunović, who, under the codename "Zdenka", served as a courier in the resistance and subsequently became his personal secretary. Haas and Tito suddenly parted company in 1943 in Jajce during the second meeting of AVNOJ after she reportedly walked in on him and Davorjanka. The last time Haas saw Broz was in 1946. Davorjanka died of tuberculosis in 1946 and Tito insisted that she be buried in the backyard of the Beli Dvor, his Belgrade residence.
Answer: {'text': ['Davorjanka Paunović'], 'answer_start': [320]}

Question: How many radio spins did Kelly Clarkson have by 2010?


**The generated answers from ChatGPT are**

**Question**: With whom did Tito have a parallel life during his marriage to Haas?

**Answer**: Tito had a parallel relationship with Davorjanka Paunović.

**Explanation**: The passage mentions that throughout Tito's relationship with Herta Haas, he maintained a promiscuous life and had a parallel relationship with Davorjanka Paunović, who served as a courier in the resistance and later became his personal secretary. This parallel relationship ultimately led to the separation of Tito and Haas in 1943.

**Question**: How many radio spins did Kelly Clarkson have by 2010?

**Answer**: By 2010, Kelly Clarkson had accumulated over four million radio spins.

**Explanation**: The context mentions that by 2010, Kelly Clarkson led the field among American Idol winners with over four million radio spins. This indicates the number of times her music had been played on the radio up to that point.

**Question**: What is the most recent standard for external drive connectivity?

**Answer**: The most recent standard for external drive connectivity is Thunderbolt.

**Explanation**: The passage mentions various standards for external drive connectivity, including USB 2.0, USB 3.0, eSATA, ExpressCard, FireWire (IEEE 1394), and Thunderbolt. The keyword "most recently" indicates that Thunderbolt is the latest standard among those listed, making it the answer to the question.

**Question**: In what year was the École Polytechnique founded?

**Answer**: 1794

**Explanation**: The context mentions that the École Polytechnique, which has educated French élites, was founded in 1794. Therefore, the answer to the question is 1794 based on this historical information.

**Question**: Why did the state House think they could change Oklahoma's motto?

**Answer**: The state House believed they could change Oklahoma's motto because a constituent researched the Oklahoma Constitution and found no "official" vote regarding the current motto "Labor Omnia Vincit," which, according to them, opened the door for adopting an entirely new motto, "Oklahoma - In God We Trust!"

**Explanation**: The context explains that the state House passed a resolution (HCR 1024) to change Oklahoma's state motto. The reasoning behind this decision is provided by the author of the resolution, who stated that a constituent's research revealed no official vote on the current motto, creating an opportunity to propose a new one.

**Question**: Who developed a far east trade and began to frequent the island?

**Answer**: The Dutch developed a Far East trade and began to frequent the island.

**Explanation**: The passage indicates that in developing their Far East trade, the Dutch started to frequent the island of Saint Helena. The Dutch involvement is highlighted in the context, along with the reasons for the decline in visits by the Portuguese and Spanish.

**Question**: Describe Gaddafi's humble upbringing.

**Answer**: Muammar Gaddafi had a humble upbringing in a tent near Qasr Abu Hadi, a rural area outside the town of Sirte in the deserts of western Libya. His family, part of a small and relatively un-influential tribal group called the Qadhadhfa, were Nomadic Bedouins, and his father earned a meager subsistence as a goat and camel herder. Gaddafi's family was illiterate, kept no birth records, and his date of birth is not known with certainty. Growing up in Bedouin culture, Gaddafi developed a preference for the desert over the city, a preference he maintained throughout his life.

**Explanation**: The context provides details about Muammar Gaddafi's upbringing, emphasizing the modest and traditional Bedouin lifestyle of his family. The information includes his birth circumstances, family background, and the influence of Bedouin culture on his personal preferences.

**Question**: What is the objective of Jehovah Witnesses' door-to-door ministry?

**Answer**: The objective of Jehovah Witnesses' door-to-door ministry is to spread their beliefs by engaging in public preaching, distributing literature in multiple languages, and initiating regular "Bible studies" with individuals who are not already members. The ultimate goal is to have the student baptized as a member of the Jehovah's Witnesses group. Witnesses are encouraged to devote as much time as possible to their ministry, and there is a reporting system in place, with baptized members required to submit an individual monthly "Field Service Report."

**Explanation**: The context explains that Jehovah Witnesses engage in door-to-door ministry as part of their efforts to spread their beliefs. The primary objective is to initiate Bible studies with non-members, leading to baptism and membership in the Jehovah's Witnesses group. The reporting system ensures accountability and emphasizes the importance of regular and active participation in the ministry.

**Question**: What is the first step to read the information encoded in a gene's DNA and produce the protein it specifies?

**Answer**: The first step to read the information encoded in a gene's DNA and produce the protein it specifies is transcription. During transcription, the gene's DNA is transcribed into messenger RNA (mRNA).

**Explanation**: The provided context explains the two steps required for gene expression. The first step is transcription, where the information encoded in a gene's DNA is transcribed into mRNA. The second step is translation, where the mRNA is used to produce the specified protein. This process is collectively known as gene expression.

**Question**: What do the sulfides convert to after the copper matte is roasted?

**Answer**: After the copper matte is roasted, the sulfides convert to oxides.

**Explanation**: The context describes the process of roasting copper matte, where the resulting copper matte (Cu2S) is subjected to roasting. During this roasting process, all sulfides present in the copper matte are converted into oxides. The specific chemical reactions involved in this conversion are not detailed in the provided context.

**Question**: In which European countries do many Russian immigrants live?

**Answer**: Many Russian immigrants live in the United Kingdom, Spain, Portugal, France, Italy, Belgium, Greece, Norway, and Austria.

**Explanation**: The context provides a list of European countries with significant Russian-speaking communities. These countries include the United Kingdom, Spain, Portugal, France, Italy, Belgium, Greece, Norway, and Austria. The information highlights the presence of Russian immigrants in these nations, contributing to diverse Russian-speaking communities.

**Question**: Who thought Burke and Fox shouldn't have made their argument public?

**Answer**: Charles Burney thought Burke and Fox shouldn't have made their argument public.

**Explanation**: In the given context, Charles Burney is mentioned as viewing Burke's Appeal as "a most admirable book" but believed that the differences in the Whig Party between Burke and Fox should not be aired publicly. Burney thought that the disagreement between Burke and Fox should have been kept within the party rather than being made public.

**Question**: What type of trace is often narrower than a ground trace?

**Answer**: A signal trace is often narrower than a ground trace.

**Explanation**: The context explains that each trace on a printed circuit board consists of a flat, narrow part of the copper foil that remains after etching. The resistance of the traces is determined by width and thickness, and it mentions that power and ground traces may need to be wider than signal traces. Therefore, signal traces are often narrower than ground traces.

**Question**: How did Gaddafi's decisions in the oil industry impact other OPEC states?

**Answer**: Gaddafi's decisions in the oil industry, particularly the move to increase the price of Libyan oil in 1969, had a broader impact on other OPEC states. Following Libya's lead, other OPEC states also increased the price of crude oil in 1970, resulting in a global rise in oil prices.

**Explanation**: The provided context explains that Gaddafi, dissatisfied with existing trade terms, successfully increased the price of Libyan oil in 1969. This move influenced other OPEC states to follow suit in 1970, contributing to a global increase in crude oil prices. Additionally, the Tripoli Agreement negotiated by the Revolutionary Command Council (RCC) with oil corporations brought significant additional revenues to Libya, showcasing the broader impact of Gaddafi's decisions on the oil industry within OPEC.

**Question**: If the hippocampus doesn't store information, what does it do?

**Answer**: While the hippocampus itself does not store information, it plays a crucial role in the consolidation of information from short-term to long-term memory. It is essential for the process of learning new information and facilitating the establishment of stable and permanent changes in neural connections distributed throughout the brain.

**Explanation**: The context explains that the hippocampus is not involved in storing information itself but is critical for the consolidation of information from short-term to long-term memory. It is particularly important for learning new information, and without it, the ability to store new memories in long-term memory is compromised. The hippocampus facilitates the process of forming stable neural connections that support long-term memory.

#####Claude#####

**Designing prompt**

Based on the following question and context, answer the question first as "**Question** : ", "**Answer** : " and then provide a brief explanation of your reasoning as "**Explanation** : ".

Question : ""

Context : ""

In [103]:
# for the given dataset
for i in select.iterrows():
  print(f"\nQuestion: {i[1]['question']}")
  print(f"Context: {i[1]['context']}")
  print(f"Answer: {i[1]['answers']}")
  # break


Question: With whom did Tito have a parallel life during his marriage to Haas?
Context: His next relationship was with Herta Haas, whom he married in 1940. Broz left for Belgrade after the April War, leaving Haas pregnant. In May 1941, she gave birth to their son, Aleksandar "Mišo" Broz. All throughout his relationship with Haas, Tito had maintained a promiscuous life and had a parallel relationship with Davorjanka Paunović, who, under the codename "Zdenka", served as a courier in the resistance and subsequently became his personal secretary. Haas and Tito suddenly parted company in 1943 in Jajce during the second meeting of AVNOJ after she reportedly walked in on him and Davorjanka. The last time Haas saw Broz was in 1946. Davorjanka died of tuberculosis in 1946 and Tito insisted that she be buried in the backyard of the Beli Dvor, his Belgrade residence.
Answer: {'text': ['Davorjanka Paunović'], 'answer_start': [320]}

Question: How many radio spins did Kelly Clarkson have by 2010?


**The generated answers from Claude 2 are**

**Question** : With whom did Tito have a parallel life during his marriage to Haas?

**Answer** : Tito had a parallel relationship with Davorjanka Paunović during his marriage to Haas.

**Explanation** : The context states that "All throughout his relationship with Haas, Tito had maintained a promiscuous life and had a parallel relationship with Davorjanka Paunović, who, under the codename "Zdenka", served as a courier in the resistance and subsequently became his personal secretary." This clearly indicates that Tito had a parallel relationship, i.e. an affair, with Davorjanka Paunović while he was married to Haas.

**Question**: How many radio spins did Kelly Clarkson have by 2010?

**Answer**: By 2010, Kelly Clarkson had over four million radio spins.

**Explanation**: The last sentence of the context states "By 2010, four winners each had more than a million radio spins, with Kelly Clarkson leading the field with over four million spins." This clearly states that by 2010, Kelly Clarkson had over four million radio spins, which is the answer to the question.

**Question**: What is the most recent standard for external drive connectivity?

**Answer**: The most recent standard for external drive connectivity is Thunderbolt.

**Explanation**: The passage lists several standards for connecting external drives, including USB, eSATA, ExpressCard, and FireWire. It states that "most recently" the Thunderbolt standard has emerged. Therefore, Thunderbolt is the most recent standard mentioned for connecting external drives.

**Question**: In what year was the École Polytechnique founded?

**Answer**: The École Polytechnique was founded in 1794.

**Explanation**: The passage states that "The oldest German Institute of Technology is the Braunschweig University of Technology (founded in 1745 as "Collegium Carolinum"). Another exception is the École Polytechnique, which has educated French élites since its foundation in 1794." This clearly states that the École Polytechnique was founded in 1794.

The context does not explicitly state why the state House thought they could change Oklahoma's motto. However, some possible reasons based on the details provided:

**Question**: Why did the state House think they could change Oklahoma's motto?

**Answer**: The House thought they could change the motto because a constituent researched the state constitution and found no "official" vote recording the adoption of the current motto "Labor Omnia Vincit". This made them believe the door was open for a new motto to be adopted.

**Explanation**: The passage states that the author of the resolution to change the motto said "a constituent researched the Oklahoma Constitution and found no "official" vote regarding "Labor Omnia Vincit", therefore opening the door for an entirely new motto." So the lack of an official vote documenting the adoption of the current motto led the House to think they could change it.

In developing their Far East trade, the Dutch also began to frequent the island.

**Question**: Who developed a far east trade and began to frequent the island?

**Answer**: The Dutch developed a far east trade and began to frequent the island.

**Explanation**: The passage states that "In developing their Far East trade, the Dutch also began to frequent the island". This clearly states that it was the Dutch who developed a far east trade and subsequently began visiting/frequenting the island often as part of that trade.

**Question:** Describe Gaddafi's humble upbringing.

**Answer:** Gaddafi was born in a tent in the Libyan desert to illiterate, nomadic Bedouin parents. His father earned a meager living herding goats and camels. They were from a small, unimportant tribe and kept no birth records. Gaddafi's early life living in Bedouin culture greatly influenced his tastes and lifestyle throughout his life, as he preferred the desert to the city.

**Explanation:** The passage details how Gaddafi was born into an impoverished desert tribe, living in tents and herding animals to survive. His parents were illiterate and unimportant. These details about his modest, simple upbringing among Bedouin nomads describe a humble beginning for Gaddafi. Key details include: born in a tent, illiterate parents, small/uninfluential tribe, father earned little as a herder, no birth records kept.

Based on the context provided, the objective of Jehovah's Witnesses' door to door ministry is:

**Question**: What is the objective of Jehovah Witnesses' door to door ministry?

**Answer**: The objective is to start a regular "Bible study" with any non-member person with the intention that the student be baptized as a member of the Jehovah's Witness group.

**Explanation**: The passage clearly states that "The objective is to start a regular "Bible study" with any person who is not already a member, with the intention that the student be baptized as a member of the group". Therefore, the goal of the door-to-door ministry is to begin Bible studies with non-members with the ultimate goal of having them get baptized and join the Jehovah's Witnesses.

The first step to read the information encoded in a gene's DNA and produce the protein it specifies is:

**Question**: What is the first step to read the information encoded in a gene's DNA and produce the protein it specifies?

**Answer**: The first step is that the gene's DNA is transcribed to messenger RNA (mRNA).

**Explanation**: The passage clearly states that "First, the gene's DNA is transcribed to messenger RNA (mRNA)." This indicates that transcription of the gene's DNA to mRNA is the first step in reading the gene's information and producing its specified protein.

**Question**: What do the sulfides convert to after the copper matte is roasted?

**Answer**: After the copper matte is roasted, the sulfides convert into oxides.

**Explanation**: The passage states that the copper matte consisting of Cu2S is roasted "to convert all sulfides into oxides". Therefore, the sulfides present in the copper matte convert into oxides after the roasting process.

Based on the context, the European countries with significant Russian immigrant populations are:

**Question**: In which European countries do many Russian immigrants live?

**Answer**: Many Russian immigrants live in the United Kingdom, Spain, Portugal, France, Italy, Belgium, Greece, Norway, and Austria.

**Explanation**: The passage clearly states "The United Kingdom, Spain, Portugal, France, Italy, Belgium, Greece, Brazil, Norway, and Austria have significant Russian-speaking communities." So these European nations have large populations of Russian-speaking immigrants, per the context provided.

Charles Burney viewed that the differences between Burke and Fox should not have been made public.

**Question**: Who thought Burke and Fox shouldn't have made their argument public?  

**Answer**: Charles Burney thought Burke and Fox should not have made their argument public.

**Explanation**: The passage states that Charles Burney "believed the differences in the Whig Party between Burke and Fox should not be aired publicly". This directly indicates that Burney thought Burke and Fox should not have publicly aired their political arguments.

**Question**: What type of trace is often narrower than a ground trace?  

**Answer**: Signal traces are often narrower than ground traces.

**Explanation**: The passage states that "Power and ground traces may need to be wider than signal traces." This comparison indicates that signal traces, which carry signals, are often narrower than ground traces, which power and ground the circuit.

By threatening to reduce production, Gaddafi was able to increase the price Libya charged for its oil. Other OPEC states then followed Libya's lead by also increasing their oil prices. So Gaddafi's decisions positively impacted other OPEC states by allowing them to also raise their oil export prices.

**Question**: How did Gaddafi's decisions in the oil industry impact other OPEC states?

**Answer**: Gaddafi's decisions to improve terms and threaten reduced production allowed Libya to increase oil prices, which other OPEC states then followed, also raising their prices and benefiting from higher oil revenue.  

**Explanation**: The passage notes that after Gaddafi took action in Libya's oil industry, "in 1970, other OPEC states followed suit, leading to a global increase in the price of crude oil." So Gaddafi's oil policies provided an example that other major oil producing states emulated to their benefit.

Based on the context, if the hippocampus doesn't store information itself, then:

**Question**: If the hippocampus doesn't store information what does it do?

**Answer**: The hippocampus is essential for consolidating information from short-term to long-term memory and changing neural connections related to new memories. However, it does not store the information itself.

**Explanation**: The passage states that the hippocampus "is essential (for learning new information) to the consolidation of information from short-term to long-term memory" and "may be involved in changing neural connections for a period of three months or more after the initial learning." However, it then clearly states that "it does not seem to store information itself." So the hippocampus enables the transfer and integration of memories, but does not store the memories.

####3. Results analysis and evaluation (30 points)####

####1) (10 points)####

The following table provides the results of comparison of the answers generated by all the three models to the ground truth, 1 being correct and 0 being incorrect response. Two of the answers are marked in decimal which indicates that it was true partially.

| Model / Sample | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 |
|--------------|---|---|---|---|---|---|---|---|---|----|----|----|----|----|----|
| RoBERTa      | 1 | 1 | 1 | 1 | 1 | 1 |.2 | 1 | 1 |  0 | 1  | 1  | 0  | 1  | 0  |
| Chat-GPT     | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 |  1 | 1  | 1  | 1  | 1  | 1  |
| Claude 2      | 1 | 1 | 1 | 1 | 1 | 1 |.5 | 1 | 1 |  1 | 1  | 1  | 1  | 1  | 1  |

For better calculation, it can be considered that 0.2 be incorrect and 0.5 be true. Based on the table above, the scores (number of data samples provided correct answers/15) for these models (out of 15) can be calculated as follows:


1.   RoBERTa  : 11
2.   Chat-GPT : 15
3.   Claude 2 : 15

####2) (10 points)####

The pre-trained RoBERTa model performed well, providing correct answers for 11 out of 15 data samples. This indicates a strong capability in understanding and responding to the given questions.

It particularly struggled in answering three questions and partially answered one question correctly. The three incorrect answers were empty strings and not wrong per se. Upon further investigation it was found that the generated answer start and answer end tokens were the reason the model was unable to generate an answer.

The answer_start_index was greater than the answer_end_index which is why the model resulted in an empty string. The reasons could be lack of training examples, complex contexts or the model parameters and can be addressed by fine-tuning the model, adjusting the parameters and employing different pre-processing methods.

**Chat-GPT** on the other hand achieved a perfect score, providing correct answers for all 15 data samples. This suggests that Chat-GPT, with its conversational and reasoning abilities, comprehensively understood and addressed the questions. Chat-GPT answered every question to the format of response asked without any deviation.

The main plus points to be noted are that it has better contextual understanding, reasoning abilities, consistency and quality in responses and human-like interactions. The success of Chat-GPT in achieving a perfect score showcases its robust performance in natural language understanding and question answering.

Similar to Chat-GPT, **Claude 2** also achieved a perfect score, indicating a high level of proficiency in answering the questions. In some samples, where the answer was unclear, Claude gave an initial message stating the ambiguity in the answer and then went ahead to answer as per the given format. Despite the ambiguity, the answers were correct. Only in one sample it did not perform at par with Chat-GPT and gave an answer with some missing points as compared to the ground truth.

Claude's answers were much more concise than Chat-GPT's but the accuracy being at par. Achieving a perfect score and demonstrating a proactive approach to handling ambiguity are positive indicators of Claude 2's capabilities.

**Advantages and Disadvantages:**

**RoBERTa:**
- **Advantages:**
  - Strong performance, providing correct answers for 11 out of 15 data samples.
  - Indicates a good capability in understanding and responding to questions.

- **Disadvantages:**
  - Struggled with three questions and partially answered one question correctly.
  - Requires fine-tuning and adjustments to address these issues.

**Chat-GPT:**
- **Advantages:**
  - Achieved a perfect score, providing correct answers for all 15 data samples.
  - Demonstrates strong contextual understanding, reasoning abilities, and consistency.
  - Shows proficiency in natural language understanding and question answering.
  - Provides human-like interactions and responds to the format of questions without deviation.

- **Disadvantages:**
  - It might face challenges in handling ambiguous questions or situations, but this is not specified in the given context.

**Claude 2:**
- **Advantages:**
  - Achieved a perfect score, indicating a high level of proficiency in answering questions.
  - Demonstrates a proactive approach to handling ambiguity by providing an initial message in unclear situations.
  - Answers are correct even in ambiguous cases.
  - Concise answers with accuracy comparable to Chat-GPT.

- **Disadvantages:**
  - Answers are more concise than Chat-GPT, which may be considered a disadvantage if more detailed responses are desired.

####3) (10 points)####

**For sample 11,**

Question: In which European countries do many Russian immigrants live?

Answer: {'text': ['The United Kingdom, Spain, Portugal, France, Italy, Belgium, Greece, Brazil, Norway, and Austria'], 'answer_start': [196]}

*Ground Truth Answer:*
- The United Kingdom, Spain, Portugal, France, Italy, Belgium, Greece, Brazil, Norway, and Austria.

*Model Answers:*
1. **Roberta:**
   - Answer: The United Kingdom, Spain, Portugal, France, Italy, Belgium, Greece, Brazil, Norway, and Austria.

2. **Chat-GPT:**
   - Answer: Many Russian immigrants live in the United Kingdom, Spain, Portugal, France, Italy, Belgium, Greece, Norway, and Austria.

3. **Claude 2:**
   - Answer: Many Russian immigrants live in the United Kingdom, Spain, Portugal, France, Italy, Belgium, Greece, Norway, and Austria.

*Comparison:*
1. **Roberta:**
   - In this case, Roberta correctly identifies all the countries mentioned in the context where Russian immigrants live. It demonstrates the model's ability to comprehend and extract relevant information accurately.

2. **Chat-GPT:**
   - Chat-GPT, while generally performing well, misses mentioning Brazil, a country listed in the ground truth. This indicates a slight deviation from the complete set of countries provided in the context. The model, however, maintains the correct structure of the answer and captures the primary information.

3. **Claude 2:**
   - Similar to Chat-GPT, Claude 2 also omits Brazil from its response. However, it correctly identifies most of the European countries mentioned in the context, showcasing a reasonable understanding of the question.

**For sample 7,**

Question: Describe Gaddafi's humble upbringing.

Answer: {'text': ['was born in a tent near Qasr Abu Hadi, a rural area outside the town of Sirte in the deserts of western Libya'], 'answer_start': [16]}

*Ground Truth Answer:*
- was born in a tent near Qasr Abu Hadi, a rural area outside the town of Sirte in the deserts of western Libya

*Model Answers:*
1. **Roberta:**
   - Answer: Nomadic Bedouins

2. **Chat-GPT:**
   - Answer: Muammar Gaddafi had a humble upbringing in a tent near Qasr Abu Hadi, a rural area outside the town of Sirte in the deserts of western Libya. His family, part of a small and relatively un-influential tribal group called the Qadhadhfa, were Nomadic Bedouins, and his father earned a meager subsistence as a goat and camel herder. Gaddafi's family was illiterate, kept no birth records, and his date of birth is not known with certainty. Growing up in Bedouin culture, Gaddafi developed a preference for the desert over the city, a preference he maintained throughout his life.

3. **Claude 2:**
   - Answer: Gaddafi was born in a tent in the Libyan desert to illiterate, nomadic Bedouin parents. His father earned a meager living herding goats and camels. They were from a small, unimportant tribe and kept no birth records. Gaddafi's early life living in Bedouin culture greatly influenced his tastes and lifestyle throughout his life, as he preferred the desert to the city.

*Comparison:*
1. **Roberta:**
   - The answer provided by Roberta does not capture the specific details mentioned in the ground truth, but captures one single detail. It lacks the information about the specific location and details of Gaddafi's upbringing.

2. **Chat-GPT:**
   - Chat-GPT provides a more detailed and accurate answer, including the specific location, tribal group, and Gaddafi's father's occupation. It aligns well with the ground truth answer.

3. **Claude 2:**
   - While Claude 2 correctly mentions Gaddafi's birth in a tent in the Libyan desert to illiterate, nomadic Bedouin parents and his father's occupation, it doesn't provide additional information about the Qadhadhfa tribe, the lack of birth records, or Gaddafi's preference for the desert over the city. Claude 2's response is more concise and doesn't delve into as much detail. While it still captures the essential information, it may be considered less comprehensive than the response from Chat-GPT.


**References**
1.   https://huggingface.co/datasets/squad_v2
2.   https://huggingface.co/docs/transformers/main/en/quicktour
3.   https://huggingface.co/docs/transformers/main/en/model_doc/roberta#transformers.RobertaForQuestionAnswering
4.   https://huggingface.co/deepset/roberta-base-squad2
5.   https://chat.openai.com/
6.   https://www.anthropic.com/index/claude-2

